#### 1. Import pandas library

In [81]:
import pandas as pd

#### 2. Import mysql.connector


In [82]:
import mysql.connector 

#### 2.2. Run the following line


In [84]:
cnx = mysql.connector.connect(user='guest', password='relational', host='relational.fit.cvut.cz', database='stats', use_pure=True)

### Import users table:

#### 3. Initialize cursor (as seen in classes' Jupyter Notebook)

In [18]:
cursor = cnx.cursor()

#### 4. Initialize query  (as seen in classes' Jupyter Notebook) 
#### You want the users table from stats database. (Hint: stats.users)

In [19]:
query = ("SELECT * FROM stats.users")

#### 5. Send query to database  (as seen in classes' Jupyter Notebook)

In [20]:
cursor.execute(query)

#### 6. Create a new dataframe (preferably called users) with the following columns names (as seen in classes' Jupyter Notebook)

In [22]:
columns_users = ['userId', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'AccountId', 'Age', 'ProfileImageUrl']


In [23]:
users_df = pd.DataFrame(columns = columns_users)
users_df

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl


#### 7. Fill the new users dataframe with the data you can get iterating through cursor (as seen in classes' Jupyter Notebook)

In [24]:
for i, data in enumerate(cursor):
    users_df.loc[i] = data

#### 8. Rename Id column to userId

In [79]:
users_df = users_df.rename(columns={"Id": "userId"})

### Import posts table:

#### 9. Initialize cursor (as seen in classes' Jupyter Notebook)

In [85]:
cursor2 = cnx.cursor()

#### 10. Initialize query  (as seen in classes' Jupyter Notebook) 
#### You want the posts table from stats database. (Hint: no hint this time ;) )

In [86]:
query2 = ("SELECT * FROM stats.posts")

#### 11. Send query to database  (as seen in classes' Jupyter Notebook)

In [87]:
cursor2.execute(query2)

#### 12. Create a new dataframe (preferably called posts) with the following columns names (as seen in classes' Jupyter Notebook)

In [32]:
columns_posts = ['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId', 'LasActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId', 'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate', 'OwnerDisplayName', 'LastEditorDisplayName']

In [ ]:
posts_df = pd.DataFrame (columns = columns_posts)
posts_df

#### 12. Fill the new posts dataframe with the data you can get iterating through cursor (as seen in classes' Jupyter Notebook)

In [88]:
#for i, data in enumerate(cursor2):
 #   posts_df.loc[i] = data
posts_df = pd.read_csv('posts_table.csv')
posts_df.head()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 13. Rename Id column to postId and OwnerUserId to userId

In [89]:
posts_df = posts_df.rename(columns={"Id": "postId", "OwnerUserId" : "userId"})

posts_df.describe()

,postId,PostTypeId,AcceptedAnswerId,Score,ViewCount,userId,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,ParentId
count,40000.000000,40000.000000,7266.000000,40000.000000,15895.000000,38962.000000,15895.000000,40000.000000,7293.000000,19725.000000,23569.000000
mean,22709.318350,1.636175,23186.498211,4.086500,1206.583202,6079.063087,1.594401,2.011525,3.360346,4635.762180,19986.577666
std,13763.536209,0.594277,14994.186353,6.589488,3872.525910,5224.896435,2.055572,2.673990,7.637988,5751.397127,13752.497021
min,1.000000,1.000000,5.000000,-19.000000,14.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,1.000000
25%,10883.750000,1.000000,10936.500000,1.000000,174.000000,1317.000000,1.000000,0.000000,1.000000,702.000000,7869.000000
50%,22050.500000,2.000000,21794.500000,2.000000,387.000000,4856.000000,1.000000,1.000000,2.000000,2116.000000,18423.000000
75%,33536.500000,2.000000,33589.250000,5.000000,978.000000,9651.000000,2.000000,3.000000,3.000000,7290.000000,30723.000000
max,48325.000000,7.000000,114363.000000,192.000000,175495.000000,55226.000000,136.000000,45.000000,233.000000,53985.000000,48314.000000


#### 14. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [112]:
users = users_df [["userId", "Reputation","Views","UpVotes","DownVotes"]].copy()
posts = posts_df [["postId", "Score","userId","ViewCount","CommentCount"]].copy()
users.dtypes


userId        object
Reputation    object
Views         object
UpVotes       object
DownVotes     object
dtype: object

#### 15. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [111]:
merged = posts.merge(users, how='inner', on="userId")
merged.dtypes

postId            int64
Score             int64
userId           object
ViewCount       float64
CommentCount      int64
Reputation       object
Views            object
UpVotes          object
DownVotes        object
dtype: object

#### 16. How many missing values do you have in your merged dataframe? On which columns?

In [92]:
merged_null = merged.isnull().sum()
print(merged_null.head())
list(merged_null [merged_null > 0].index)

postId              0
Score               0
userId              0
ViewCount       23572
CommentCount        0
dtype: int64


['ViewCount']

#### 17. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [106]:
null_displ = merged[(merged['ViewCount'].isnull()==True)]

null_displ = null_displ[['postId', "userId" ,"ViewCount", "Views" ,"UpVotes" ,"DownVotes"]]
null_displ.tail(20)

merged_nonull = merged.dropna(axis=0, how='any')
merged_nonull.head()

# I would drop them due to View counts being a very imprtatn metric for engagement and advertisement and i would want to have a 
# clean data set. I would not that the anlysis woudl be wrong due to seeing that certain posts had average views when 
# we could not confirm that was trues due to nulls.

,postId,Score,userId,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1,23,8,1278.0,1,6764,1089,604,25
2,36,41,8,67396.0,7,6764,1089,604,25
6,114,30,8,1220.0,1,6764,1089,604,25
9,168,17,8,1022.0,1,6764,1089,604,25
10,170,56,8,7733.0,1,6764,1089,604,25


#### 18. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [108]:
merged_nonull.dtypes

postId            int64
Score             int64
userId           object
ViewCount       float64
CommentCount      int64
Reputation       object
Views            object
UpVotes          object
DownVotes        object
dtype: object

In [114]:
new_merged = merged_nonull.astype({"Score":"float64",'userId': 'int64', 'ViewCount': 'int64', 'Reputation' : 'int64',"Views": 'int64', "UpVotes": 'int64', "DownVotes": 'int64'})
new_merged.dtypes

postId            int64
Score           float64
userId            int64
ViewCount         int64
CommentCount      int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
dtype: object